The intention of this project is to train a NER model on anonymised clinical data obtained from an official source, annotated by trained medical professionals.
Instead the CANTEMIST competition dataset will be used, which is a collection of clinical notes in Spanish, originally annotated with the use BRAT in .ann format. 
Time and budgetary constraints meant that the data used will be but annotated with the use of GPT-4 API in the file annotateGPT_bg.ipynb to identify the symptoms, conditions, findings and tests in the clinical notes. The training data is small at the moment for demonstration purposes, with intentions to expand this with professional annotations in the future.

Spacy will be used to train the NER model based on the GPT-4 annotated data. The model will be trained on the training set and demonstrated. 
The evaluation is planned to be done in the future, based on F1 score and precision. The model will be used to extract the entities from the clinical notes, and subsequently in the future train a machine learning model to predict the occurence of cancer from the entities (symptoms, conditions, findings etc.)

Set up the environment:

In [1]:
# %pip install -U spacy
# !python3 -m spacy download es_core_news_md

The data needing to be used for development should be moved to final_data/ folder. The data should be in the format of .txt and .ann files. The .txt files should contain the clinical notes, and the .ann files should contain the annotations in BRAT format. 

Load the model and the training data

In [27]:
import spacy
from find_entity import find_start_end

# es_core_news_md - medium-sized model for Spanish
nlp = spacy.load("es_core_news_md")
# nlp = spacy.blank("es") for blank

path = "./final_data/"
# the folder in the variable path contains the .txt and .ann files for the training data
# we need to import them and the entities like (Text, {entities: [(start, end, label)]})

# get entities from .ann file and text from .txt file
def get_entities(text_path, annotation_path):
    with open(text_path, "r", encoding='utf-8') as file:
        text = file.read()
    with open(annotation_path, "r") as file:
        annotations = file.read().splitlines()
    entities = []
    for annotation in annotations:
        split = annotation.split()
        if annotation[0] == "T" and len(split) > 4: # why is this necessary? because the .ann file contains other lines
            # e.g. annotation.split() == ['T1', 'SYMPTOM', '0', '9', 'Anisocoria']
            _, label, _, _, *entity = split # *entity is the rest of the list
            joined_entity = ' '.join(entity)
            # now we need to search the text for the entity (only because we worked with GPT, if not then can just say _, label, start, end, *entity)
            start, end = find_start_end(text.lower(), joined_entity.lower())
            entities.append((int(start), int(end), label, joined_entity))
    return (text, {"entities": entities})


In [28]:
# get entities from all .ann and .txt files in a folder
import os
from spacy.tokens import DocBin
from tqdm import tqdm

# this will get the data from all the .txt and .ann files in a folder
def get_data(folder):
    data = []
    fol = os.listdir(folder)
    for file in fol:
        if file.endswith(".txt"):
            text_path = os.path.join(folder, file)
            annp = file.replace(".txt", ".ann")
            annotation_path = os.path.join(folder, annp)
            print(annotation_path, annp in fol)
            if annp in fol:
                print("OK")
                data.append(get_entities(text_path, annotation_path))
    return data

training_data = get_data("./llm_data/")

db = DocBin() # create a DocBin object which will store the training examples
for text, annotations in tqdm(training_data):
    doc = nlp(text)
    # debug notes: char_span only works if it is in a token and the annotations must be wrong
    ents = []
    starts_ends = [] # to check for conflicts
    for obj in annotations["entities"]:
        start = obj[0]
        end = obj[1]
        label = obj[2]
        span = doc.char_span(start, end, label=label)
        if span is None:
            # print("Skipping entity", span)
            # print("Error")
            continue
        else:
            # print(f"Adding entity: {span.text} [{start}-{end}, {label}]")
            # check for conflicts
            conflicts = False
            for s,e in starts_ends:
                # check for conflicts and overlaps
                # (start >= s and end <= e) or (start <= s and end >= e)
                if ((start < e and end>s)):
                    conflicts = True
            if conflicts or start == -1:
                print(f"Conflict detected with span: {span.text} [{start}-{end}, {label}]")
            else:
                ents.append(span)
                starts_ends.append((start, end))
    doc.ents = ents
    db.add(doc)
db.to_disk("./final_data_retrain.spacy") # save the DocBin object

./llm_data/caso_clinico_radiologia581.ann True
OK
./llm_data/cc_onco29.ann True
OK
./llm_data/cc_onco609.ann True
OK
./llm_data/cc_onco184.ann True
OK
./llm_data/cc_onco580.ann True
OK
./llm_data/cc_onco224.ann True
OK
./llm_data/caso_clinico_radiologia621.ann True
OK
./llm_data/caso_clinico_radiologia43.ann True
OK
./llm_data/cc_onco146.ann True
OK
./llm_data/cc_onco608.ann True
OK
./llm_data/cc_onco28.ann True
OK
./llm_data/caso_clinico_radiologia580.ann True
OK
./llm_data/caso_clinico_radiologia582.ann True
OK
./llm_data/caso_clinico_radiologia41.ann True
OK
./llm_data/cc_onco144.ann True
OK
./llm_data/caso_clinico_radiologia623.ann True
OK
./llm_data/cc_onco596.ann False
./llm_data/caso_clinico_radiologia622.ann True
OK
./llm_data/cc_onco186.ann True
OK
./llm_data/cc_onco145.ann True
OK
./llm_data/caso_clinico_radiologia40.ann True
OK
./llm_data/caso_clinico_radiologia583.ann True
OK
./llm_data/caso_clinico_radiologia44.ann True
OK
./llm_data/caso_clinico_radiologia50.ann True
OK
.

  2%|▏         | 4/162 [00:00<00:17,  9.23it/s]

Conflict detected with span: mama derecha [347-359, ANATOMICAL]
Conflict detected with span: mama derecha [347-359, ANATOMICAL]


 15%|█▍        | 24/162 [00:02<00:09, 13.85it/s]

Conflict detected with span: fosa iliaca derecha [70-89, ANATOMICAL]


...

 95%|█████████▌| 154/162 [00:13<00:01,  7.47it/s]

Conflict detected with span: adalimumab [477-487, PROCEDURE]


 97%|█████████▋| 157/162 [00:13<00:00,  7.49it/s]

Conflict detected with span: años [22-26, QUANTITY]
Conflict detected with span: hepatoblastoma embrionario con diferenciación colangiolar [541-598, CONDITION]
Conflict detected with span: hepatoblastoma embrionario [541-567, CONDITION]


100%|██████████| 162/162 [00:14<00:00, 11.50it/s]


To visualise the DocBin object, we can use the displacy module from spacy.

In [29]:
from spacy import displacy

# Convert the DocBin object to a list of Doc objects
docs = list(db.get_docs(nlp.vocab))

# Iterate over the Doc objects and visualize them using displacy
for doc in docs:
    displacy.render(doc, style="ent", jupyter=True)

# serve the visualisation for purposes of the report
# displacy.serve(doc, style="ent", auto_select_port=True, host="localhost")

/opt/homebrew/lib/python3.11/site-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [30]:
# export to a csv file for viewing
import csv
with open("train_data.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(["text", "entities"])
    writer.writerows(training_data)
# open as a preview of the file to demonstrate the training data

NER pipeline component

In [34]:
print(nlp.pipe_names) # 'ner' is already at the end of our pipeline for the model
ner = nlp.get_pipe("ner")
print(ner.labels) # the labels that the model can predict

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
('ANATOMICAL', 'BACKGROUND', 'CONDITION', 'DEVICE', 'DRUG', 'FINDING', 'LOC', 'LOCATION', 'MISC', 'ORG', 'PER', 'PROCEDURE', 'QUANTITY', 'STAGE', 'SYMPTOM', 'TEST', 'TIME', 'TREATMENT')


Add the entity labels to the model from the bg-train.spacy file

In [32]:
# Load the DocBin object from the bg-train.spacy file
db = DocBin().from_disk("./final_data_retrain.spacy")

# Iterate over the Doc objects in the DocBin
for doc in db.get_docs(nlp.vocab):
    for ent in doc.ents:
        ner.add_label(ent.label_)


In [33]:
# we now have the new labels in the NER pipeline
labels = ner.labels
print(labels)

('ANATOMICAL', 'BACKGROUND', 'CONDITION', 'DEVICE', 'DRUG', 'FINDING', 'LOC', 'LOCATION', 'MISC', 'ORG', 'PER', 'PROCEDURE', 'QUANTITY', 'STAGE', 'SYMPTOM', 'TEST', 'TIME', 'TREATMENT')


Train the NER model - just with ner component

In [35]:
!python3 -m spacy train config.cfg --output ./output --paths.train ./final_data_retrain.spacy --paths.dev ./final_data_retrain.spacy 

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     88.50    0.00    0.00    0.00    0.00
  1     200       6697.35  18162.81    8.32   17.89    5.42    0.08
  2     400      17263.69  10185.77    9.38   23.75    5.85    0.09
  3     600      11174.85   7670.26   15.24   32.42    9.96    0.15
  4     800       3468.23   6696.34   24.76   34.77   19.23    0.25
  6    1000       2769.43   6021.45   31.83   40.37   26.27    0.32
  7    1200       3150.40   5719.91   39.12   48.12   32.96    0.39
  8    1400       3940.03   5558.86   47.80   50.78   45.15    0.48
  9    1600       6332.85   513

In [37]:
my_nlp = spacy.load(r"./output/model-best") 

files = ["./spacy-train/cc_onco3.txt"]
with open(files[0], "r", encoding='utf-8') as file:
    text = file.read()
    
    doc = my_nlp(text) 
    # spacy.displacy.render(doc, style="ent", jupyter=True) 
    # spacy.displacy.serve(doc, style="ent")
    displacy.serve(doc, style="ent",auto_select_port=True, host="localhost")


FileNotFoundError: [Errno 2] No such file or directory: './spacy-train/cc_onco3.txt'

references
https://towardsdatascience.com/train-ner-with-custom-training-data-using-spacy-525ce748fab7
